In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
# Load data from JSON file
file_path = 'C:/Users/HP/Desktop/angles_output_085347.json'  # Replace with your file path

data = []
with open(file_path) as f:
    for line in f:
        data.append(json.loads(line))

In [3]:
# Convert JSON data to DataFrame
df = pd.DataFrame(data)

In [4]:
# Separate features and labels
X = df.drop('Label', axis=1).values
y = df['Label'].values

In [5]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [6]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [7]:
# Shape of each dataset
print("Number of samples in:")
print(f"Training data (X_train): {X_train.shape[0]}")
print(f"Testing data (X_test): {X_test.shape[0]}")

Number of samples in:
Training data (X_train): 788
Testing data (X_test): 197


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

In [9]:
# Reshape data for LSTM: (samples, timesteps, features)
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [10]:
# Build LSTM model
lstm_model = Sequential()
lstm_model.add(Input(shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
lstm_model.add(LSTM(50))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [11]:
# Compile model
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Train model
lstm_history = lstm_model.fit(X_train_lstm, y_train, epochs=50, batch_size=32, validation_data=(X_test_lstm, y_test))

Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.2998 - loss: 1.5741 - val_accuracy: 0.5178 - val_loss: 1.3495
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5581 - loss: 1.3260 - val_accuracy: 0.6802 - val_loss: 1.2385
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6667 - loss: 1.2262 - val_accuracy: 0.7005 - val_loss: 1.1176
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6954 - loss: 1.1203 - val_accuracy: 0.7259 - val_loss: 1.0532
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7107 - loss: 1.0726 - val_accuracy: 0.7513 - val_loss: 0.9915
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7333 - loss: 1.0068 - val_accuracy: 0.7360 - val_loss: 0.9460
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7490 - loss: 0.9568 - val_accuracy: 0.7868 - val_loss: 0.8953
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7864 - loss: 0.9015 - val_accuracy: 0.8071 - val_loss

In [13]:
# Save the history
with open('training_history.json', 'w') as f:
    json.dump(lstm_history.history, f)

In [14]:
# Save LSTM model
lstm_model.save('new_verification_lstm_model_hip_incorrect_reason_json.h5')

In [15]:
from tensorflow.keras.models import load_model

# Load LSTM model
lstm_model = load_model('new_verification_lstm_model_hip_incorrect_reason_json.h5')

In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import mediapipe as mp
import math

# Define acceptable ranges for each angle (adjust these values based on your requirements)
ANGLE_THRESHOLDS = {
    'left_elbow_angle': (60, 180),
    'right_elbow_angle': (60, 180),
    'left_shoulder_angle': (30, 160),
    'right_shoulder_angle': (30, 160),
    'left_knee_angle': (70, 180),
    'right_knee_angle': (70, 180),
    'left_hip_angle': (50, 180),
    'right_hip_angle': (50, 180)
}

# Function to extract joint coordinates from the image
def extract_angles_from_image(image_path):
    # Initialize Mediapipe Pose model
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()

    # Load the image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform pose estimation
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Extract keypoints
        landmarks = results.pose_landmarks.landmark

        # Get coordinates
        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
        left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
        left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
        left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
        right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

        # Calculate angles
        left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        left_shoulder_angle = calculate_angle(left_elbow, left_shoulder, left_hip)
        right_shoulder_angle = calculate_angle(right_elbow, right_shoulder, right_hip)
        left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
        left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
        right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)

        return {
            'left_elbow_angle': left_elbow_angle,
            'right_elbow_angle': right_elbow_angle,
            'left_shoulder_angle': left_shoulder_angle,
            'right_shoulder_angle': right_shoulder_angle,
            'left_knee_angle': left_knee_angle,
            'right_knee_angle': right_knee_angle,
            'left_hip_angle': left_hip_angle,
            'right_hip_angle': right_hip_angle
        }
    else:
        return None

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    angle = math.degrees(math.atan2(c[1] - b[1], c[0] - b[0]) - math.atan2(a[1] - b[1], a[0] - b[0]))
    angle = abs(angle)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Function to predict the correctness of the pose and identify issues
def predict_pose_correctness(image_path, model):
    angles = extract_angles_from_image(image_path)
    if angles is None:
        print("No pose landmarks detected.")
        return None

    angles_array = np.array(list(angles.values())).reshape(1, 1, 8)  # Reshape to (1, 1, 8) for LSTM input
    prediction = model.predict(angles_array)
    correctness = prediction[0][0] > 0.5  # Binary classification threshold

    if correctness:
        return "yes"
    else:
        incorrect_parts = []
        for angle_name, angle_value in angles.items():
            min_threshold, max_threshold = ANGLE_THRESHOLDS[angle_name]
            if not (min_threshold <= angle_value <= max_threshold):
                incorrect_parts.append(f"{angle_name} is out of range ({angle_value:.2f}°)")

        if incorrect_parts:
            return f"No, issues found: {', '.join(incorrect_parts)}"
        else:
            return "No, but no specific angle issues detected."

# Example usage
if __name__ == "__main__":
    loaded_model = load_model('new_verification_lstm_model_hip_incorrect_reason_json.h5')
    image_path = 'C:/Users/HP/Desktop/test_12.jfif'
    result = predict_pose_correctness(image_path, loaded_model)

    if result is not None:
        print(f"The pose in '{image_path}' is predicted to be {result}.")

C:\Users\HP\anaconda3\envs\mediapipe\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 410ms/step
The pose in 'C:/Users/HP/Desktop/test_12.jfif' is predicted to be No, issues found: left_shoulder_angle is out of range (167.87°), right_shoulder_angle is out of range (171.83°).
